<a href="https://colab.research.google.com/github/YaraAnusha/mini-project/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install cdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 KB 1.3 MB/s eta 0:00:00


In [ ]:
import os,glob
import cdflib
import numpy as np
from tqdm import tqdm



In [ ]:
def fillValue(array, windowSize, threshold):
  


   for row in tqdm(range(len(array))):
       for col in range(len(array[row])):
           if array[row][col] < threshold:
               minCol = max(0, col - windowSize // 2)
               maxCol = min(len(array[row]) - 1, col + windowSize // 2)
               sum = 0
               num = 0
               for idx in range(minCol, maxCol + 1):
                   if array[row][idx] > threshold:
                       sum += array[row][idx]
                       num += 1



               try:
                   array[row][col] = sum/num
               except:
                   print(f"num = {num}")



   return array.T



In [ ]:
def preprocessing(folderSet, keySet, storeFolder, year, windowSize, threshold):
   



   for folder, keys in zip(folderSet, keySet):
       files = sorted(glob.glob(os.path.join(folder, "*_"+str(year) + '*.cdf')))



       for key in keys:
           subList = []
      
           for file in files:
               print(file)
               cdf_pycdf = cdflib.cdf_to_xarray(file)
               try:
                   subList.append(cdf_pycdf[key][:])
               except Exception as e:
                   print(f"error key {e}")
                   break



           subList = np.concatenate(subList, axis = 0)
           np.save(os.path.join(storeFolder, str(year) + "-mag-" + key + ".npy"), subList, allow_pickle=True, fix_imports=True)
           if key == 'B1GSE' or key == 'BGSE':
               subList = fillValue(subList.transpose(), windowSize, threshold)
           np.save(os.path.join(storeFolder, "filled-"+ str(year) + "-mag-" + key + ".npy"), subList, allow_pickle=True, fix_imports=True)
           del subList



In [ ]:
if __name__ == "__main__":
   storeFolder = "./npyFiles"
   os.makedirs(storeFolder, exist_ok=True)



   folderMAG = "/content/dscovr-h0-mag"
   folderMFI = "/content/wind-mfi-mfi_h2"
   folderSWE = "/content/wind-swe-swe_h1"
   folderSet = [folderMAG, folderMFI, folderSWE]
   magKey = ['Epoch1', 'B1GSE']
   mfiKey = ['Epoch', 'BGSE']
   sweKey = ['Proton_Np_moment', 'Proton_V_moment', 'Proton_W_moment']
   keySet = [magKey, mfiKey, sweKey]



   windowSize = 10
   threshold = -1000000



   start_year = 2021
   end_year = 2021



   for year in range(start_year, end_year+1):



       preprocessing(folderSet, keySet, storeFolder, year, windowSize, threshold)



ValueError: ignored